<a href="https://colab.research.google.com/github/shabeeth2/chatbot-RL-Phi3-vision/blob/main/Phi_3_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Requirements

requirements_content = """accelerate==0.30.1
flash_attn==2.5.8
numpy==1.24.4
Pillow==10.1.0
Requests==2.31.0
transformers==4.40.2
gradio==4.29.0
pydub==0.25.1
edge-tts
# gradio==4.15.0
# Pillow==10.3.0
# torch==2.3.0
# torchvision==0.18.0
"""

In [3]:
#@title Install
import os
root_path=os.getcwd()
# root_path="/content"
base_path=f"{root_path}"

with open(f"{base_path}/requirements.txt", "w") as file:
    file.write(requirements_content)

print(f"File '{base_path}/requirements.txt' has been successfully created with the specified contents.")

!pip install -r $base_path/requirements.txt
from IPython.display import clear_output
clear_output()
import time
time.sleep(6)

In [ ]:
#@ kill and cancel the restart
import os
os.kill(os.getpid(), 9)

In [1]:
#@title <-- Play the audio { display-mode: "form" }

%%html
<b>Press play on the music player to keep the tab alive, then run the cell below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [1]:
#@title Download /microsoft/Phi-3-vision-128k-instruct
from tqdm import tqdm
import urllib
import os
import shutil
def conditional_download(url, download_file_path):
    print(f"Downloading {os.path.basename(download_file_path)}")
    base_path = os.path.dirname(download_file_path)

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if os.path.exists(download_file_path):
        os.remove(download_file_path)

    try:
        request = urllib.request.urlopen(url)  # type: ignore[attr-defined]
        total = int(request.headers.get('Content-Length', 0))
    except urllib.error.URLError as e:
        print(f"Error: Unable to open the URL - {url}")
        print(f"Reason: {e.reason}")
        return

    with tqdm(total=total, desc='Downloading', unit='B', unit_scale=True, unit_divisor=1024) as progress:
        try:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))  # type: ignore[attr-defined]
        except urllib.error.URLError as e:
            print(f"Error: Failed to download the file from the URL - {url}")
            print(f"Reason: {e.reason}")
            return

    print(f"Download successful!")
    print(f"URL: {url}")
    print(f"Save at: {download_file_path}")


#set up base_path
import os
root_path=os.getcwd()
# root_path="/content"
base_path=f"{root_path}"



download_dir=f"{base_path}/microsoft/Phi-3-vision-128k-instruct"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
download_list = [
    "config.json",
    "configuration_phi3_v.py",
    "image_embedding_phi3_v.py",
    "image_processing_phi3_v.py",
    "model.safetensors.index.json",
    "modeling_phi3_v.py",
    "preprocessor_config.json",
    "processing_phi3_v.py",
    "sample_inference.py",
    "special_tokens_map.json",
    "tokenizer.json",
    "tokenizer_config.json",
    "model-00001-of-00002.safetensors",
    "model-00002-of-00002.safetensors"
]
base_url="https://huggingface.co/microsoft/Phi-3-vision-128k-instruct/resolve/main"
fix_url="https://raw.githubusercontent.com/neuralfalcon/Phi-3-vision-128k-instruct-colab/main/"
for i in download_list:
  if i=="config.json" or i=="modeling_phi3_v.py":
    file_url=f"{fix_url}/{i}"
  else:
    file_url=f"{base_url}/{i}"
  download_file_path=f"{download_dir}/{i}"
  conditional_download(file_url, download_file_path)
from IPython.display import clear_output
clear_output()

In [2]:
#@title import /microsoft/Phi-3-vision-128k-instruct
import os
root_path=os.getcwd()
# root_path="/content"
base_path=f"{root_path}"
from PIL import Image
import requests
from transformers import AutoModelForCausalLM
from transformers import AutoProcessor
import os
model_id = f"{base_path}/microsoft/Phi-3-vision-128k-instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation="eager")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
#@title Main code
temp_image="https://www.google.com/imgres?q=cat&imgurl=https%3A%2F%2Fcdn.britannica.com%2F34%2F235834-050-C5843610%2Ftwo-different-breeds-of-cats-side-by-side-outdoors-in-the-garden.jpg&imgrefurl=https%3A%2F%2Fwww.britannica.com%2Fanimal%2Fcat&docid=Bvzzy2OOLWm60M&tbnid=ryYDG4V7pxKPVM&vet=12ahUKEwj05Ymo6t-GAxUsyzgGHSe9DXIQM3oECBgQAA..i&w=1600&h=1054&hcb=2&ved=2ahUKEwj05Ymo6t-GAxUsyzgGHSe9DXIQM3oECBgQAA"
def ask_question(question="What is shown in this image?",user_image=""):
  global temp_image
  if os.path.exists(user_image):
    image = Image.open(user_image)
    temp_image=user_image
  elif os.path.exists(temp_image):
    image = Image.open(temp_image)
  else:
    return "Please upload a image"

  messages = [
      {"role": "user", "content": f"<|image_1|>\n{question}"},
      # {"role": "assistant", "content": "The chart displays the percentage of respondents who agree with various statements about their preparedness for meetings. It shows five categories: 'Having clear and pre-defined goals for meetings', 'Knowing where to find the information I need for a meeting', 'Understanding my exact role and responsibilities when I'm invited', 'Having tools to manage admin tasks like note-taking or summarization', and 'Having more focus time to sufficiently prepare for meetings'. Each category has an associated bar indicating the level of agreement, measured on a scale from 0% to 100%."},
      # {"role": "user", "content": "Provide insightful questions to spark discussion."}
  ]

  # url = "https://assets-c4akfrf5b4d3f4b7.z01.azurefd.net/assets/2024/04/BMDataViz_661fb89f3845e.png"
  # image = Image.open(requests.get(url, stream=True).raw)


  prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

  inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0")

  generation_args = {
      "max_new_tokens": 500,
      "temperature": 0.0,
      "do_sample": False,
  }

  generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args)

  # remove input tokens
  generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
  response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

  # print(response)
  return response.strip()


In [16]:
question = 'what is the color of the cat?'  # @param {type: "string"}
user_image = '/content/image_cat.jpeg'  # @param {type: "string"}
answer=ask_question(question,user_image)
answer

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


'The cat has a striped pattern with shades of brown, black, and white.'

In [17]:
#@title Use as gradio chatbot
import gradio as gr
import time

def add_message(history, message):
    if "files" in message and message["files"]:
        for file in message["files"]:
            history.append(((file,), None))
    if "text" in message and message["text"] is not None:
        history.append((message["text"], None))
    return history, gr.update(value=None, interactive=True)

def bot(history):
    # response = "**That's cool!**"

    # Extract the last file path and last message
    last_file_path = None
    last_message = None
    for entry in reversed(history):
        if last_file_path is None and isinstance(entry[0], tuple):
            last_file_path = entry[0][0]
        if last_message is None and isinstance(entry[0], str):
            last_message = entry[0]
        if last_file_path and last_message:
            break

    if last_file_path:
        print("Last file path:", last_file_path)
        user_image=last_file_path
    else:
        user_image=""
    if last_message:
        print("Last message:", last_message)
        question=last_message
    else:
        question="What is shown in this image?"

    response=ask_question(question,user_image)
    history.append(("", ""))
    for character in response:
        history[-1] = (history[-1][0], history[-1][1] + character)
        time.sleep(0.05)
        yield history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(
        [],
        elem_id="chatbot",
        bubble_full_width=False
    )

    chat_input = gr.MultimodalTextbox(
        interactive=True, file_types=["image"], placeholder="Enter message or upload file...", show_label=False
    )

    chat_msg = chat_input.submit(add_message, [chatbot, chat_input], [chatbot, chat_input])
    bot_msg = chat_msg.then(bot, chatbot, chatbot, api_name="bot_response")
    bot_msg.then(lambda: gr.update(interactive=True), None, [chat_input])

demo.queue()
demo.launch(debug=False)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://345c4ccdab91a90599.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
